# Value Iteration
> Find policy that performs well in grid_world

## Step 1: Imports

In [1]:
from grid_world import Grid

In [2]:
# I have my own implementation differents from src
def print_grid(g):
    print("="*(g.width*16+1))
    for i in range(g.height):
        print("|", end="")
        for j in range(g.width):
            p = ", ".join(g.actions.get((i,j), " "))
            print("{:^15}|".format(p), end="")
        print()
        if i < g.height-1:
            print("-"*(g.width*16+1))
    print("="*(g.width*16+1))
def print_results(V, P, g):
    print("="*(g.width*16+1))
    for i in range(g.height):
        print("|", end="")
        for j in range(g.width):
            if not g.actions[(i, j)] and (i, j) not in g.rewards:
                print("{:^15}|".format("(X)"), end="")
            else:
                v = g.rewards[(i, j)] if (i, j) in g.rewards else V.get((i, j), 0)
                p = P.get((i, j), " ")
                p = "T" if (i, j) in g.rewards else p
                print("{:^15}|".format("{:1.2f} ({})".format(v, p)), end="")
        print()
        if i < g.height-1:
            print("-"*(g.width*16+1))
    print("="*(g.width*16+1))

## Step 2: initialization

In [3]:
# Grid
rewards = {
    (0, 3): 1,
    (1, 3): -1 }
actions = {
    (0, 0): ('D', 'R'),
    (0, 1): ('R', 'L'),
    (0, 2): ('R', 'L', 'D'),
    (0, 3): (),
    (1, 0): ('U', 'D'),
    (1, 1): (),
    (1, 2): ('U', 'D', 'R'),
    (1, 3): (),
    (2, 0): ('U', 'R'),
    (2, 1): ('R', 'L'),
    (2, 2): ('U', 'R', 'L'),
    (2, 3): ('U', 'L') }
grid = Grid(4, 3, (2, 0))
grid.set(rewards, actions)
print_grid(grid)

# V, gamma, threshold
V = {s: 0 for s in grid.all_states()}
gamma = 0.9
SMALL_ENOUGH = 1e-3

|     D, R      |     R, L      |    R, L, D    |               |
-----------------------------------------------------------------
|     U, D      |               |    U, D, R    |               |
-----------------------------------------------------------------
|     U, R      |     R, L      |    U, R, L    |     U, L      |


## Step 3: Main Function

In [4]:
def policy_improvment(g, V):
    action_value_functions = {}
    for s in grid.all_states():
        action_value_functions[s] = {}
        for a in grid.actions[s]:
            grid.set_state(s)
            reward = grid.move(a)
            action_value_functions[s][a] = (reward + gamma * V[grid.current_state()])
    policy = {}
    for s in grid.all_states():
        if action_value_functions[s]:
            policy[s] = max(action_value_functions[s], key=action_value_functions[s].get)
    return policy
while True:
    biggest_change = 0
    for s in grid.all_states():
        old_v = V[s]
        action_values = {}
        for a in grid.actions[s]:
            grid.set_state(s)
            reward = grid.move(a)
            new_v = (reward + gamma * V[grid.current_state()])
            action_values[a] = new_v
        new_v = max(action_values.values()) if action_values else 0
        V[s] = new_v
        biggest_change = max(biggest_change, abs(old_v - new_v))
    if biggest_change < SMALL_ENOUGH:
        break
    policy = policy_improvment(grid, V)

## Step 4: Show Result

In [5]:
print_results(V, policy, grid)

|   0.81 (R)    |   0.90 (R)    |   1.00 (R)    |   1.00 (T)    |
-----------------------------------------------------------------
|   0.73 (U)    |      (X)      |   0.90 (U)    |   -1.00 (T)   |
-----------------------------------------------------------------
|   0.66 (U)    |   0.73 (R)    |   0.81 (U)    |   0.73 (L)    |
